In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_kan
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/transliterated/'

    train_data = read_task(dataset_location , split = 'kan_train_trans')
    dev_data = read_task(dataset_location , split = 'kan_dev_trans')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_kan
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Kannada' + 'transliterated' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.9808406233787537
DEV ACC: 0.61251495215311
DEV Precision: 0.4960656792261362
DEV Recall: 0.4582119535169774
DEV F1Score: 0.43142322088616614
BEST ACCURACY -->  DEV: 0.61251
BEST PRECISION -->  DEV: 0.49607
BEST RECALL -->  DEV: 0.45821
BEST F1SCORE -->  DEV: 0.43142
TIME PER EPOCH: 4.325381489594777

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.8726581335067749
DEV ACC: 0.6395783492822966
DEV Precision: 0.48613578194798285
DEV Recall: 0.4720

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0978587865829468
DEV ACC: 0.6082535885167464
DEV Precision: 0.45183665603096834
DEV Recall: 0.43537805313373495
DEV F1Score: 0.4231497863109672
BEST ACCURACY -->  DEV: 0.60825
BEST PRECISION -->  DEV: 0.45184
BEST RECALL -->  DEV: 0.43538
BEST F1SCORE -->  DEV: 0.42315
TIME PER EPOCH: 4.336327290534973

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.2095025777816772
DEV ACC: 0.6309808612440192
DEV Precision: 0.47170307940488065
DEV Recall: 0.4

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.108222246170044
DEV ACC: 0.6026465311004784
DEV Precision: 0.4953620694030819
DEV Recall: 0.4207219431266891
DEV F1Score: 0.41429687526715303
BEST ACCURACY -->  DEV: 0.60265
BEST PRECISION -->  DEV: 0.49536
BEST RECALL -->  DEV: 0.42072
BEST F1SCORE -->  DEV: 0.4143
TIME PER EPOCH: 4.710701835155487

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0910874605178833
DEV ACC: 0.6205143540669856
DEV Precision: 0.48484131233435934
DEV Recall: 0.4642

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.9194063544273376
DEV ACC: 0.6119916267942584
DEV Precision: 0.4441327621850664
DEV Recall: 0.44546592741779906
DEV F1Score: 0.4090925097178768
BEST ACCURACY -->  DEV: 0.61199
BEST PRECISION -->  DEV: 0.44413
BEST RECALL -->  DEV: 0.44547
BEST F1SCORE -->  DEV: 0.40909
TIME PER EPOCH: 4.719046560923259

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0220463275909424
DEV ACC: 0.6309808612440192
DEV Precision: 0.47200633688636784
DEV Recall: 0.50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0918220281600952
DEV ACC: 0.62813995215311
DEV Precision: 0.4922704572369889
DEV Recall: 0.4876855793063928
DEV F1Score: 0.46112370180254325
BEST ACCURACY -->  DEV: 0.62814
BEST PRECISION -->  DEV: 0.49227
BEST RECALL -->  DEV: 0.48769
BEST F1SCORE -->  DEV: 0.46112
TIME PER EPOCH: 4.750871082146962

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.7290956974029541
DEV ACC: 0.6508672248803827
DEV Precision: 0.5077688600451054
DEV Recall: 0.53681

In [6]:
print(best_dev_acc)

0.6735944976076556


In [7]:
print(best_epoch)

9


In [8]:
print(best_dev_precision)

0.5670852558799365


In [9]:
print(best_dev_recall)

0.5589823859412939


In [10]:
print(best_dev_f1score)

0.538255334714048


In [11]:
print(overall_list_dev_acc)

[[0.61251495215311, 0.6395783492822966, 0.6660436602870813, 0.6527362440191388, 0.640924043062201, 0.6575209330143541, 0.6100478468899522, 0.6389802631578948, 0.6527362440191388, 0.6361393540669856], [0.6082535885167464, 0.6309808612440192, 0.6550538277511961, 0.6438397129186603, 0.6309808612440192, 0.6385316985645934, 0.6342703349282297, 0.6522876794258373, 0.6196172248803827, 0.6282147129186603], [0.6026465311004784, 0.6205143540669856, 0.65938995215311, 0.6650717703349283, 0.6404754784688995, 0.6422697368421052, 0.6081040669856459, 0.6214862440191388, 0.6295604066985646, 0.6318779904306221], [0.6119916267942584, 0.6309808612440192, 0.6575956937799042, 0.6532595693779905, 0.6550538277511961, 0.64376495215311, 0.6475777511961722, 0.6167763157894737, 0.6276166267942584, 0.627168062200957], [0.62813995215311, 0.6508672248803827, 0.6735944976076556, 0.6651465311004784, 0.6623056220095693, 0.6498953349282297, 0.6338965311004784, 0.6471291866028708, 0.6329246411483254, 0.6357655502392344]]

In [12]:
print(overall_list_dev_precision)

[[0.4960656792261362, 0.48613578194798285, 0.5519962558081754, 0.5163909678369292, 0.5129778910928644, 0.5151497857844146, 0.528764221791593, 0.5488822526034954, 0.6027078883715421, 0.535193980130745], [0.45183665603096834, 0.47170307940488065, 0.4943979054395427, 0.49760016135473495, 0.49896577673350817, 0.5403763640665555, 0.5227549608192725, 0.5502038462913076, 0.5414538543749774, 0.5558144723353702], [0.4953620694030819, 0.48484131233435934, 0.5177716925903318, 0.5127368674965775, 0.518813573532825, 0.5152840362072151, 0.5000095574360279, 0.5350323253457929, 0.543314295705233, 0.553859014925719], [0.4441327621850664, 0.47200633688636784, 0.5017318134300263, 0.5806241620905285, 0.5440003457858088, 0.5189391038254674, 0.5471727648465616, 0.518867342433343, 0.5423402973406491, 0.5607621699765661], [0.4922704572369889, 0.5077688600451054, 0.5312169696957201, 0.5068080150609, 0.5374509429221997, 0.5670852558799365, 0.5084555135892035, 0.5534502395999724, 0.5581989657932523, 0.5577402798

In [13]:
print(overall_list_dev_recall)

[[0.4582119535169774, 0.47205225589918104, 0.5418977641412908, 0.539210710639081, 0.5214124310150555, 0.5017121618725897, 0.5057422144419049, 0.5274046067607847, 0.5533484458125607, 0.5106123027221394], [0.43537805313373495, 0.46142611326074245, 0.5169496779665123, 0.5286176683731567, 0.5295685791520297, 0.5394447030437743, 0.5215789130030617, 0.5251048605052263, 0.5654769731863033, 0.5243305694368177], [0.4207219431266891, 0.4642109493221229, 0.532615878273773, 0.5241290125423252, 0.5380405401194173, 0.523060465252451, 0.5173318463129889, 0.5133679723660022, 0.5398614086838825, 0.5182394620499742], [0.44546592741779906, 0.5085208783773377, 0.5286056650217218, 0.5531361419524591, 0.5373377549288753, 0.5444021435377332, 0.5466447377012392, 0.5020563984284784, 0.5390169307816367, 0.5334255871407573], [0.4876855793063928, 0.5368151831086323, 0.5332070770529467, 0.5232431844041752, 0.5424913799182023, 0.5454558298598545, 0.5134798720293372, 0.5490945941899361, 0.5413015945105024, 0.5589823

In [14]:
print(overall_list_dev_f1score)

[[0.43142322088616614, 0.45111408342744147, 0.5205983906215506, 0.5067643382560234, 0.4957888865947347, 0.4894348718771106, 0.49511631694632074, 0.5156135518523521, 0.5467250752448024, 0.5032070998074075], [0.4231497863109672, 0.4458755948759769, 0.4856499422586104, 0.4896317949844384, 0.4911712388221703, 0.5122515591117902, 0.4888367766727079, 0.515902702336926, 0.5253934443704175, 0.5194738196459959], [0.41429687526715303, 0.4451970712969953, 0.5041351579169319, 0.4993445079208657, 0.5029988093840284, 0.5008455631272754, 0.4817666311534303, 0.49952583614449964, 0.5195665713193148, 0.5119135865231134], [0.4090925097178768, 0.4687554783094186, 0.4983961425721199, 0.5368237967692532, 0.5199157113961502, 0.5063067987039719, 0.5221612808131457, 0.47382812755504405, 0.5156324096367478, 0.5170818911009887], [0.46112370180254325, 0.4941402009220237, 0.5043110377138126, 0.4946720821517292, 0.515540998055502, 0.5320254343371243, 0.49548381854820156, 0.5257470238365612, 0.5219940515188809, 0.53

In [15]:
#The best model is 0